This notebook illustrates the code used in the paper ```U. Dobramysl, D. Holcman, Mixed analytical-stochastic simulation method for the recovery of a Brownian gradient source from probability fluxes to small windows, Journal of Computational Physics 355 (2018)```.

To run this notebook, you need a Python 3 installation (e.g. Anaconda) and the python packages `cython`, `numpy`, `scipy`, `matplotlib`, `seaborn`, `multiprocess` and `tqdm`. To install them, simply run:

```pip install cython numpy scipy matplotlib seaborn multiprocess tqdm```

In [ ]:
%matplotlib inline
import numpy as np
from numpy.random import normal, uniform, poisson
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from utils import json_cache_write, json_cache_read, send_parallel_jobs
import pyximport; pyximport.install()
from receptor_fluxes_strip import SimulationDomain
rc("figure", figsize=(8,6))
sns.set_context("poster")
sns.set_style("ticks"); None

# Simulation of particle fluxes to receptors at steady state - disk in a strip

In [ ]:
def run_simulation(parameters):
    """Perform a single realisation of the simulation.
    
    Args:
        source_distance (float): Distance to the source x.
        thetas (list|floats): Receptor angles.
        nr_particles (int): Number of particles to release.
    Returns:
        Receptor fluxes.
    """
    import scipy.stats
    from receptor_fluxes_strip import SimulationDomain

    params = {
        "strip_width": 3.0,
        "nr_particles": 1e2,
    }
    params.update(parameters)
    
    x, y = params["source_position"]
    d = (x**2 + y**2)**0.5
    if abs(y) > params["strip_width"] / 2. or d < 1.0:
        return {"parameters": params,
                "results": {}}
    
    sd = SimulationDomain(params["thetas"], source_position=params["source_position"],
                          strip_width=params["strip_width"])
    for _ in range(int(params["nr_particles"])):
        sd.release_particle()
    return {"parameters": params, "results": {"normalized_receptor_fluxes": sd.receptor_fluxes()[1].tolist()}}

## Simulation Results

In [ ]:
def plot_fluxes(name, fix=[], legend=None, axlabel="", labelpos=0.3, exclude=[]):
    from collections import defaultdict
    from scipy.interpolate import interp1d
    from matplotlib.lines import Line2D
    data = defaultdict(lambda : defaultdict(list))
    widths = set()
    dists = set()
    for res in json_cache_read(name):
        params = res["parameters"]
        fluxes = res["results"]["normalized_receptor_fluxes"]
        key = (params['source_position'][0], params['strip_width'])
        if key[0] != 5. and key[0] !=50:
            dists.add(key[0])
            widths.add(key[1])
            data[key][params['thetas'][1]].append(fluxes[1])

    styles = {}
    colors = ['k', 'b', 'b' , 'b']
    templ = ['-', '--', '-.', ':', '-', '--']
    for i, L in enumerate(sorted(dists)):
        styles[L] = {'color': colors[i], 'style': templ[i]}
    maxf = 0.0
    minf = 1.0
    if not isinstance(labelpos, list):
        labelpos = len(data)*[labelpos]
    for j, x in enumerate(sorted(data)):
        d = data[x]
        theta = sorted(d)
        fluxes = [np.mean(d[t]) for t in theta]
        maxf = max(max(fluxes), maxf)
        minf = min(min(fluxes), minf)
        for f in fix:
            fluxes[f] = float("NaN")
        l = plt.plot(theta, fluxes, color=styles[x[0]]['color'], label="$L=%g,a=%g$" % x, ls=styles[x[0]]['style'])[0]
    maxt = max(theta)
    plt.xlabel(r"Receptor 2 angle $\theta_2$")
    plt.ylabel(r"Splitting probability $J_2/(J_1+J_2)$")
    plt.xticks([0, np.pi/4, np.pi/2, np.pi*3/4., np.pi, np.pi*5/4., np.pi*3/2., np.pi*7/4., 2*np.pi],
               ["$0$", r"$\pi/4$", r"$\pi/2$", r"$3\pi/4$", r"$\pi$", r"$5\pi/4$", r"$3\pi/2$", r"$7\pi/8$", r"$2\pi$"])
    plt.axis(xmax=1*maxt, ymin=min(0.45, 0.95*minf), ymax=max(0.51, maxf))
    if legend is not None:
        handles = [Line2D([0, 1], [0, 0], ls=styles[L]['style'], color=styles[L]['color']) for L in sorted(dists)]
        labels = ["$L=%g$" % L for L in sorted(dists)]
        plt.legend(handles, labels, loc=legend)
    plt.text(-0.2, 1.05, axlabel, transform=plt.gca().transAxes)
    plt.axis(ymin=0, ymax=1)

### Receptor 1 at $\theta_1=0$, vary $\theta_2$

In [ ]:
for a in [2.1, 3.0, 5.0]:
    for L in [1.3, 2.0, 10.0, 100.0]:
        todo += 10*[{"source_position": (L, 0.0), "thetas":[0, angle], "strip_width": a, 'nr_particles': 1e3} for angle in np.linspace(0, np.pi, 50)]
results = send_parallel_jobs("strip_zero_pi_flux", run_simulation, todo)
plt.title(r"Simulation data for $\theta_1=0$")
plot_fluxes("strip_zero_pi_flux")

### Receptor 1 at $\theta_1=\pi$, vary $\theta_2$

In [ ]:
for a in [2.1, 3.0, 5.0]:
    for L in [1.3, 2.0, 10.0, 100.0]:
        todo += 10*[{"source_position": (L, 0.0), "thetas":[np.pi, angle], "strip_width": a, 'nr_particles': 1e4} for angle in np.linspace(0, np.pi, 50)]
results = send_parallel_jobs("strip_pi_flux", todo)
plt.title(r"Simulation data for $\theta_1=\pi$")
plot_fluxes("strip_pi_flux")

### Receptor 1 at $\theta_1=\pi/2$, vary $\theta_2$

The receptor fluxes below show that direction sensing with maximal separation of receptors is possible below a distance of $L=10R$ with a flux difference threshold of $5\%$. The plots of the analytical function below is in excellent agreement (If we modify Casper's expression slightly!).

In [ ]:
from random import shuffle
todo = []
for a in [2.1, 3.0, 5.0]:
    for L in [1.3, 2.0, 10.0, 100.0]:
        todo += 10*[{"source_position": (L, 0.0), "thetas":[np.pi/2., angle], "strip_width": a, 'nr_particles': 1e4} for angle in np.linspace(0, 2*np.pi, 50)]
results = send_parallel_jobs("strip_pi_half_flux", todo)
plt.title(r"Simulation data for $\theta_1=\pi/2$")
plot_fluxes("strip_pi_half_flux")

### Receptors at $\theta_1=0$, vary $\theta_2$, vary strip width $a$

In [ ]:
todo = sum([[{"source_position": (10.0, 0.0), "thetas":[0, angle], "strip_width": x, 'nr_particles': 1e1} for angle in np.linspace(0, np.pi, 10)]
            for x in [5.0, 10.0, 20.0]], [])*10000
results = send_parallel_jobs("strip_zero_pi_flux_varywidth", todo)
plt.title(r"Simulation data for $\theta_1=0$")
plot_fluxes_stripwidth("strip_zero_pi_flux_varywidth")

### Receptors at $\theta_1=\pi/2$, vary $\theta_2$, vary strip width $a$

In [ ]:
todo = sum([[{"source_position": (10.0, 0.0), "thetas":[np.pi/2., angle], "strip_width": x, 'nr_particles': 1e1} for angle in np.linspace(0, 2*np.pi, 20)]
            for x in [5.0, 10.0, 20.0]], [])*10000
results = send_parallel_jobs("strip_pihalf_flux_varywidth", todo)
plt.title(r"Simulation data for $\theta_1=\pi/2$")
plot_fluxes_stripwidth("strip_pihalf_flux_varywidth")

### Receptors at $\pi$, vary $\theta_2$, vary strip width $a$

In [ ]:
todo = sum([[{"source_position": (10.0, 0.0), "thetas":[np.pi, angle], "strip_width": x, 'nr_particles': 1e1} for angle in np.linspace(0, np.pi, 10)]
            for x in [5.0, 10.0, 20.0]], [])*10000
results = send_parallel_jobs("strip_pi_flux_varywidth", todo)
plt.title(r"Simulation data for $\theta_1=\pi$")
plot_fluxes_stripwidth("strip_pi_flux_varywidth")